In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
issue = pd.read_csv("data/fb_22_output_multi.csv.gz")

In [3]:
issue.head(2)

,text,id,ISSUE10,ISSUE11,ISSUE12,ISSUE13,ISSUE14,ISSUE15,ISSUE16,ISSUE17,...,ISSUE105,ISSUE106,ISSUE111,ISSUE118,ISSUE200,ISSUE208,ISSUE210,ISSUE212,ISSUE218,ISSUE221
0,',x_4971277182972491__ad_creative_link_descripti...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,' YOUR SOURCE FOR @GANDERNEWSROOM And I very m...,x_605605801303782__aws_ocr_text_vid,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# Function to get column names where value is 1 for the last 65 columns
def get_active_features(row):
    active_features = issue.columns[-65:][row.iloc[-65:] == 1]
    return ','.join(active_features)

In [5]:
# Apply the function to each row
issue['issue'] = issue.apply(get_active_features, axis=1)

In [6]:
# Split ids
issue['id'] = issue['id'].str.split('|')
# "Un-deduplicate", or "Re-hydrate", in WMP lingo
issue = issue.explode('id')
# Split into ad id and field
issue_ids = issue['id'].str.split('__', expand = True)
issue_ids.columns = ['ad_id', 'field']
issue = pd.concat([issue, issue_ids], axis = 1)
issue = issue.drop(labels = ['id'], axis = 1)
issue = issue[['ad_id', 'field', 'issue']]

In [ ]:
issue.rename(columns={'field': 'issue_field', 'issue': 'issue_class'}, inplace=True)

In [ ]:
# Replace various representations of missing values with NaN
missing_values = ['', None, 'NA', 'missing']
issue['issue_class'] = issue['issue_class'].replace(missing_values, np.nan)
issue = issue.dropna(subset=['issue_class'])

In [ ]:
agg_issue = issue.groupby('ad_id').agg({'issue_field': lambda x: '|'.join(x),
                                       'issue_class': lambda x: '|'.join(x)}).reset_index()

In [ ]:
agg_issue.head(5)

In [ ]:
agg_issue.shape

In [ ]:
agg_issue.to_csv("data/fb2022_output_processed.csv", index = False)